In [1]:
# Install Model maker
!pip install tflite-model-maker

In [3]:
# Imports and check that we are using TF2.x
import numpy as np
import os
import pandas as pd

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [25]:
# Download the dataset as a CSV and store as data_file
import os
data_file = os.path.abspath('spam_dataset.csv')

In [24]:
data_file.head()

AttributeError: 'str' object has no attribute 'head'

In [26]:
# Use a model spec from model maker. Options are 'mobilebert_classifier', 'bert_classifier' and 'average_word_vec'
# The first 2 use the BERT model, which is accurate, but larger and slower to train
# Average Word Vec is kinda like transfer learning where there are pre-trained word weights
# and dictionaries
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

In [29]:
# Load the CSV using DataLoader.from_csv to make the training_data
data = DataLoader.from_csv(
      filename=data_file,
      text_column='text', 
      label_column='label', 
      model_spec=spec,
      delimiter=',',
      shuffle=True,
      is_training=True)

train_data, test_data = data.split(0.9)

In [30]:
# Build the model
model = text_classifier.create(train_data, model_spec=spec, epochs=50, validation_data=test_data)

145/145 [==============================] - 2s 7ms/step - loss: 0.5896 - accuracy: 0.7114 - val_loss: 0.5003 - val_accuracy: 0.7124
Epoch 2/2
145/145 [==============================] - 1s 5ms/step - loss: 0.4054 - accuracy: 0.8280 - val_loss: 0.3264 - val_accuracy: 0.8822
Epoch 3/3
145/145 [==============================] - 1s 5ms/step - loss: 0.2698 - accuracy: 0.9134 - val_loss: 0.2194 - val_accuracy: 0.9228
Epoch 4/4
145/145 [==============================] - 1s 4ms/step - loss: 0.1943 - accuracy: 0.9325 - val_loss: 0.1670 - val_accuracy: 0.9402
Epoch 5/5
145/145 [==============================] - 1s 5ms/step - loss: 0.1555 - accuracy: 0.9386 - val_loss: 0.1393 - val_accuracy: 0.9556
Epoch 6/6
145/145 [==============================] - 1s 5ms/step - loss: 0.1319 - accuracy: 0.9472 - val_loss: 0.1206 - val_accuracy: 0.9556
Epoch 7/7
145/145 [==============================] - 1s 5ms/step - loss: 0.1168 - accuracy: 0.9517 - val_loss: 0.1080 - val_accuracy: 0.9595
Epoch 8/8
145/145 [====

In [31]:
loss, accuracy = model.evaluate(train_data)

146/146 [==============================] - 0s 2ms/step - loss: 0.0215 - accuracy: 0.9959


In [32]:
# This will export to TFLite format with the model only. 
# if you see a .json file in this directory, it is NOT the JSON model for TFJS
# See below for how to generate that.
# Please note that if you run this cell to create the tflite model then the 
# export to TFJS will fail. You'll need to rerun the model training first
model.export(export_dir='/mm_spam')
# If you want the labels and the vocab, for example for iOS, you can use this
model.export(export_dir='/mm_spam/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

# You can find your files in colab by clicking the 'folder' tab to the left of
# this code window, and then navigating 'up' a directory to find the root
# directory listing -- and from there you should see /mm_spam/

In [33]:
# Use this section for export to TFJS
# Please note that if you run the above cell to create the tflite model then the 
# export to TFJS will fail. You'll need to rerun the model training first
model.export(export_dir="/mm_js/", export_format=[ExportFormat.TFJS, ExportFormat.LABEL, ExportFormat.VOCAB])

AttributeError: 'TextClassifier' object has no attribute 'get_output_details'

In [34]:
# Optional extra
# You can use this cell to export details for projector.tensorflow.org
# Where you can explore the embeddings that were learned for this dataset
embeddings = model.model.layers[0]
weights = embeddings.get_weights()[0]
tokenizer = model.model_spec.vocab

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word in tokenizer:
  #word = tokenizer.decode([word_num])
  value = tokenizer[word]
  embeddings = weights[value]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [35]:
get_input_details()

NameError: name 'get_input_details' is not defined